In [3]:
import traci
import csv

sumo_config = "Buszka.sumocfg"

traci.start(["sumo", "-c", sumo_config, "--step-length", "1.0"])

traffic_light_id = "240977613"  

stats = []
step = 0

current_phase = traci.trafficlight.getPhase(traffic_light_id)
phase_start_step = 0
vehicles_passed_in_phase = 0

while step < 3600:  # liczba kroków symulacji
    traci.simulationStep()
    new_phase = traci.trafficlight.getPhase(traffic_light_id)

    controlled_lanes = traci.trafficlight.getControlledLanes(traffic_light_id)

    vehicles_this_step = 0
    for lane in controlled_lanes:
        vehicles_this_step += traci.lane.getLastStepVehicleNumber(lane)

    if new_phase != current_phase:
        phase_duration = step - phase_start_step
        stats.append({
            "phase": current_phase,
            "duration": phase_duration,
            "vehicles_passed": vehicles_passed_in_phase
        })
        vehicles_passed_in_phase = 0
        phase_start_step = step
        current_phase = new_phase

    vehicles_passed_in_phase += vehicles_this_step

    step += 1

traci.close()

with open("traffic_light_stats.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["phase", "duration", "vehicles_passed"])
    writer.writeheader()
    for row in stats:
        writer.writerow(row)


In [4]:
import traci
import csv

sumo_config = "Buszka.sumocfg"
traffic_light_id = "240977613"

traci.start(["sumo", "-c", sumo_config, "--step-length", "1.0"])

step = 0
current_phase = traci.trafficlight.getPhase(traffic_light_id)
phase_start_step = 0
vehicles_passed_in_phase = {}

controlled_lanes = traci.trafficlight.getControlledLanes(traffic_light_id)

def get_vehicles_on_lanes(lanes):
    vehicles_on_lanes = {}
    for lane in lanes:
        vehicles_on_lanes[lane] = set(traci.lane.getLastStepVehicleIDs(lane))
    return vehicles_on_lanes

vehicles_at_phase_start = get_vehicles_on_lanes(controlled_lanes)

stats = []

while step < 3600:
    traci.simulationStep()
    new_phase = traci.trafficlight.getPhase(traffic_light_id)

    if new_phase != current_phase:
        vehicles_at_phase_end = get_vehicles_on_lanes(controlled_lanes)

        vehicles_passed_in_phase = {}
        for lane in controlled_lanes:
            start_vehicles = vehicles_at_phase_start.get(lane, set())
            end_vehicles = vehicles_at_phase_end.get(lane, set())
            passed = start_vehicles - end_vehicles
            vehicles_passed_in_phase[lane] = len(passed)

        phase_duration = step - phase_start_step
        stats.append({
            "phase": current_phase,
            "duration": phase_duration,
            "vehicles_passed_per_lane": vehicles_passed_in_phase
        })

        vehicles_at_phase_start = vehicles_at_phase_end
        phase_start_step = step
        current_phase = new_phase

    step += 1

traci.close()

with open("traffic_light_stats_detailed.csv", "w", newline="") as f:
    fieldnames = ["phase", "duration"] + list(controlled_lanes)
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in stats:
        row_to_write = {
            "phase": row["phase"],
            "duration": row["duration"],
        }
        row_to_write.update(row["vehicles_passed_per_lane"])
        writer.writerow(row_to_write)
